In [20]:
"""

This is legacy code written to showcase how data were merged with Python 2.7

Sources of the data being combined are explained in arXiv paper number 1711.08083
https://arxiv.org/abs/1711.08083


"""




import pandas as pd
import os

os.getcwd()


a = pd.read_csv("patients' LSOA when registering for GP.csv", low_memory=False)
b = pd.read_csv("LSOA-level IMD scores.csv", low_memory=False)

merged1 = a.merge(b, how='outer', on='lsoa', indicator=True)
# merged2 = a.merge(b, on='lsoa')
# merged3 = a.merge(b, on='ods', how='right')

merged1.to_csv("merged_outer.csv", index=False)
# merged2.to_csv('merged_inner.csv', index=False)

print "ok"

1. Merging patient register numbers organised according to LSOA codes and LSOA-level IMD scores
- 751588 unique GP-LSOA pairs
- 32844 Index of Multiple Deprivation recordings for LSOAs

Results of merging:
# - 747898 records had both: a) a GP-LSOA pair, and b) IMD score for the LSOA
# - 3625 records lacked a GP-LSOA pair (this data gap was assumed there were no registered patients from those LSOA areas)
# - 269 records lacked an IMD score for an LSOA



koniec


In [67]:
# Compute:
# - average IMD for each GP
# - total number of patients
# - number of patients without known LSOA registered at GP practices

import csv
import os

if os.path.isfile("gp_imd+patients.csv"):
    os.remove("gp_imd+patients.csv")

klucze = {}
with open('merged_outer LSOA & IMD.csv','rb') as dok:
    reader = csv.reader(dok, delimiter=',')
    for i,v in enumerate(reader):
        if i > 0:
            if v[-1] == 'both': 
                if v[0] in klucze:
                    lsoa = klucze[v[0]][-3]
                    lsoa.extend([v[6],v[7]])
                    imd_kobiety = int(v[4]) * int(v[-2])
                    imd_faceci = int(v[3]) * int(v[-2])
                    klucze[v[0]][1] = int(klucze[v[0]][1]) + int(v[4]) #kobiety
                    klucze[v[0]][2] = int(klucze[v[0]][2]) + int(v[3]) #faceci
                    klucze[v[0]][3] = int(klucze[v[0]][3]) + imd_kobiety #imd kobiety
                    klucze[v[0]][4] = int(klucze[v[0]][4]) + imd_faceci #imd faceci
                else:
                    imd_kobiety = int(v[4]) * int(v[-2])
                    imd_faceci = int(v[3]) * int(v[-2])
                    lsoa = [v[6],v[7]]
                    info = [v[1],v[4],v[3],imd_kobiety,imd_faceci, lsoa,0,0]
                    klucze[v[0]] = info
            else: # co jeśli wiersz jest wybrakowany
                if v[0] in klucze:
                    lsoa = klucze[v[0]][-3]
                    lsoa.extend(['nan','nan'])

                    klucze[v[0]][6] = int(klucze[v[0]][6]) + int(v[4])
                    klucze[v[0]][7] = int(klucze[v[0]][7]) + int(v[3])
                else:
                    lsoa = ['nan','nan']
                    info = [v[1],0,0,0,0,lsoa,v[4],v[3]]
                    klucze[v[0]] = info

with open("gp_imd+patients.csv", "ab") as dok:
    writer = csv.writer(dok, delimiter = ",") 
    tytuly = ['gp_ods','gp_name','women','men','f_imd','m_imd','f_without_imd','m_without_imd','avg_imd','patient_count_all']
    writer.writerow(tytuly)
    for klucz in klucze:
        listka = [] 
        listka.append(klucz)
        listka.extend(klucze[klucz])
        ppol = listka[:6]
        dpol = listka[7:]
        listka = ppol
        listka.extend(dpol)
        if licznik < 1:
            print listka
#         licznik = licznik + 1
        pacjenci = int(klucze[klucz][2]) + int(klucze[klucz][1])
        imd = int(klucze[klucz][3]) + int(klucze[klucz][4])
        pacjenci_inni = int(klucze[klucz][-1]) + int(klucze[klucz][-2])
        sr_imd = 'nan'
        if pacjenci != 0:
            sr_imd = float(imd)/float(pacjenci)
        else:
            pass
        listka.append(sr_imd)
        pacjenci_wszyscy = pacjenci + pacjenci_inni
        listka.append(pacjenci_wszyscy)
        writer.writerow(listka)


print 'ok'

koniec


In [1]:
# Combining merged file also with fiscal records from 2015/16 of NHS payments to GP practices.
# (so, the payments are for period 2014/2015, the fiscal year in NHS ends in September)
# Index of Multiple Deprivation scores per LSOA come from year 2012.
# patient register numbers per LSOA and GP practice come from July 2015


# 7748 GPs had records both (a) of NHS payments, and (b) had IMD data
# 93 GPs only had data in dataset (a)
# 53 GPs only had data in dataset (b)

import pandas as pd
import os

os.getcwd()


a = pd.read_csv("nhspaymentsgp-15-16-csv.csv", low_memory=False)
b = pd.read_csv("gp_imd+patients.csv", low_memory=False)

merged1 = a.merge(b, how='outer', on='ods', indicator=True)
# merged2 = a.merge(b, on='lsoa')
# merged3 = a.merge(b, on='ods', how='right')

merged1.to_csv("gp_imd+patients+nhspaym.csv", index=False)
# merged2.to_csv('polaczone_inner.csv', index=False)




In [2]:
# adding in comments dataset already processed with topic modelling
# see 'aa step3.2 (optional) add_GP_ODS_codes_to_data (github).ipynb' for example of how to
# add ODS codes to the records

import pandas as pd
import os

os.getcwd()


a = pd.read_csv("regdata+ODS.csv", low_memory=False)
b = pd.read_csv("gp_imd+patients+nhspaym.csv", low_memory=False)

merged1 = a.merge(b, how='outer', on='ods', indicator=True)

merged1.to_csv("gp_imd+patients+nhspaym+feedback.csv", index=False)
# merged2.to_csv('polaczone_inner.csv', index=False)



In [31]:
# drop comments with missing data, compute month numbers (a combination of year and month)

import csv
import os
import pandas as pd

a = pd.read_csv("gp_imd+patients+nhspaym+feedback.csv", low_memory=False)
a = pd.DataFrame.dropna(a)
x = (a['year'] - 2012)*12-12+a['month']
a['period'] = x
a.to_csv('combdata_with_periods.csv')
print 'ok'

koniec


In [2]:
# final dataset merger
# the reviews data come from "regdata" object produced in R, enriched with ODS codes
# 144142 reviews were represented in both in datasets (a) and (b)
# 6 reviews were only in dataset (a) (the comment texts were very short and so LDA model skipped those reviews)
# 2246 reviews were only in dataset (b) (due to some metadata about commented-on GP practices missing)

import pandas as pd
import os

os.getcwd()


a = pd.read_csv("combdata_with_periods.csv", low_memory=False)
b = pd.read_csv("reviews.csv", low_memory=False)

merged1 = a.merge(b, how='outer', on='my.ref.num', indicator=True)

merged1.to_csv("combined_data.csv", index=False)



